# 벡터 유사도

## 1. 코사인 유사도

Similarity = cos(세타) = A•B / ||A|| ||B||

In [2]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
#코사인 유사도 계산
def cos_sim(A: np.array ,B: np.array):
    return dot(A,B)/(norm(A)*norm(B))

문서1 : 저는 사과 좋아요  
문서2 : 저는 바나나 좋아요  
문서3 : 저는 바나나 좋아요 저는 바나나 좋아요  

In [21]:
import pandas as pd

docs = [
  '저는 사과 좋아요',
  '저는 바나나 좋아요',
  '저는 바나나 좋아요 저는 바나나 좋아요'
] 
#전체 단어 집합 생성
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
vocab

['바나나', '사과', '저는', '좋아요']

In [60]:
#DTM 구하기
DTM=[]
for doc in docs:
    vector=[]
    for voca in vocab:
        #각 단어에 대해 문장에서 몇번 등장하는지 계산(TF)
        vector.append(doc.count(voca))
    DTM.append(vector)
df = pd.DataFrame(DTM,['문서1','문서2','문서3'],columns= vocab)
df

,바나나,사과,저는,좋아요
문서1,0,1,1,1
문서2,1,0,1,1
문서3,2,0,2,2


In [71]:
doc1=DTM[0]
doc2=DTM[1]
doc3=DTM[2]

In [64]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


cos_sim(1,2)와 cos_sim(1,3)의 유사도가 같음  
-> 문서의 길이가 다른 상황에서도 공정한 비교 가능  
벡터의 크기가 아닌, 방향(패턴)에 초점을 두기 때문

## 2. 코사인 유사도를 이용한 추천 시스템 구현하기

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [35]:
data = pd.read_csv('data/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [37]:
#상위 20000개의 데이터 사용
data = data.head(20000)

In [38]:
#NULL값 확인
data['overview'].isnull().sum()

135

In [41]:
# overview에서 Null값 제거(빈 값(empty value)으로 대체)
data['overview'] = data['overview'].fillna('')

In [43]:
tfidf = TfidfVectorizer(stop_words='english')
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20000, 47487)


In [44]:
#코사인 유사도 구하기
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [48]:
#타이틀과 인덱스를 갖는 테이블 생성
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [47]:
idx = indices['Father of the Bride Part II']
print(idx)

4


In [57]:
# 가장 overview가 비슷한 10개의 영화를 찾아내는 함수
def get_recommendations(title: str, cosine_sim = cosine_sim):
    #선택한 영화의 인덱스를 받아옵니다.
    idx = indices[title]
    
    #모든 영화에 대해 해당 영화와의 유사도 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #유사도에 따라 영화들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #가장 유사한 10개의 영화 선택하고 인덱스 받아오기
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    
    #제목 리턴
    return data['title'].iloc[movie_indices]

In [58]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

## 3. 유클리드 거리(Euclidean distance)

두 점 사이의 거리

In [59]:
import numpy as np
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

In [72]:
doc1=DTM[0]
doc2=DTM[1]
doc3=DTM[2]
docQ = np.array((1,1,0,1))

In [73]:
print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

1.4142135623730951
1.4142135623730951
2.6457513110645907


값이 작을수록 거리가 가깝다 -> 유사하다

## 2. 자카드 유사도(Jaccard similarity)

J(A,B) = |AnB| / |AuB| (합집합에서 교집합의 비율, 동일하면 1, 전부 다르면 0)

In [76]:
# 다음과 같은 두 개의 문서가 있습니다.
# 두 문서 모두에서 등장한 단어는 apple과 banana 2개.
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [79]:
#합집합
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(len(union),union)

12 {'like', 'passport', 'everyone', 'love', 'card', 'you', 'banana', 'coupon', 'watch', 'likey', 'holder', 'apple'}


In [83]:
#교집합
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(len(intersection),intersection)

2 {'banana', 'apple'}


In [84]:
print(len(intersection)/len(union))

0.16666666666666666


## 3. 레벤슈타인 거리(Levenshtein Distance)